# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import folium
from IPython.display import IFrame

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,pont rouge,48.8833,-72.0825,17.888,95,100,2.27,CA,1701209469
1,1,port-aux-francais,-49.3500,70.2167,42.404,98,100,10.90,TF,1701209469
2,2,adamstown,-25.0660,-130.1015,73.616,86,69,6.62,PN,1701209470
3,3,port lincoln,-34.7333,135.8667,61.484,81,61,7.56,AU,1701209183
4,4,sitka,57.0531,-135.3300,42.692,81,40,0.00,US,1701209273


In [3]:
unique_count = city_data_df['City'].nunique()
print(unique_count)

596


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
m = folium.Map(location=[0, 0], zoom_start=2)

for index, row in city_data_df.iterrows():
    city_name = row['City']
    lat = row['Lat']
    lng = row['Lng']
    humidity = row['Humidity']
    
    folium.CircleMarker(
        location=[lat, lng],
        radius=humidity/5,  
        popup=f'{city_name}, Humidity: {humidity}%',
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(m)

m.save('city_humidity_map.html')

IFrame(src='city_humidity_map.html', width=800, height=600)


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
# Filter the DataFrame based on your weather conditions
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] >= 20) & (city_data_df['Max Temp'] <= 35)
        ]

# Drop rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Reset the index after filtering
ideal_weather_df.reset_index(drop=True, inplace=True)

# Display sample data
ideal_weather_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,7,bethel,41.3712,-73.4140,33.674,51,0,4.12,US,1701209473
1,19,iqaluit,63.7506,-68.5145,29.930,92,100,9.26,CA,1701209481
2,22,grytviken,-54.2811,-36.5092,31.064,65,15,4.59,GS,1701209482
3,25,albany,42.6001,-73.9662,29.588,65,54,0.45,US,1701209484
4,33,whitehorse,60.7161,-135.0538,23.774,67,20,2.06,CA,1701209195


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''



# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,bethel,US,41.3712,-73.4140,51,
1,iqaluit,CA,63.7506,-68.5145,92,
2,grytviken,GS,-54.2811,-36.5092,65,
3,albany,US,42.6001,-73.9662,65,
4,whitehorse,CA,60.7161,-135.0538,67,


In [7]:
unique_count = hotel_df['City'].nunique()
print(unique_count)

58


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000  # 10,000 meters (10 kilometers)

# Create an empty column "Hotel Name" in the hotel_df DataFrame
hotel_df['Hotel Name'] = ''

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Set parameters for the Geoapify API request
    params = {
        "type": "accommodation",  # Search for accommodation places
        "radius": radius,
        "limit": 1,  # Limit the result to the first hotel found
        "apiKey": geoapify_key  # Use your Geoapify API key from api_keys.py
    }
    
    # Set the location parameter to specify latitude and longitude
    params["location"] = f"{lat},{lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Remove rows where "Hotel Name" is "No hotel found"
hotel_df = hotel_df[hotel_df['Hotel Name'] != "No hotel found"]

# Reset the index after removing rows
hotel_df.reset_index(drop=True, inplace=True)

# Display the updated hotel_df DataFrame
hotel_df.head()


Starting hotel search
bethel - nearest hotel: No hotel found
iqaluit - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
whitehorse - nearest hotel: No hotel found
gornyak - nearest hotel: No hotel found
rikaze - nearest hotel: No hotel found
jamestown - nearest hotel: No hotel found
akrehamn - nearest hotel: No hotel found
hamilton - nearest hotel: No hotel found
bridgewater - nearest hotel: No hotel found
klyuchi - nearest hotel: No hotel found
bodo - nearest hotel: No hotel found
sault ste. marie - nearest hotel: No hotel found
isafjordur - nearest hotel: No hotel found
ilulissat - nearest hotel: No hotel found
stanley - nearest hotel: No hotel found
selfoss - nearest hotel: No hotel found
fyodorovka - nearest hotel: No hotel found
haukipudas - nearest hotel: No hotel found
beloyarskiy - nearest hotel: No hotel found
harstad - nearest hotel: No hotel found
kiruna - nearest hotel: No hotel found
kalisz - nearest hotel: No h

,City,Country,Lat,Lng,Humidity,Hotel Name


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display


radius = 10000  # 10,000 meters (10 kilometers)


hotel_df['Hotel Name'] = ''


print("Starting hotel search")


for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Set parameters for the Geoapify API request
    params = {
        "type": "accommodation",  # Search for accommodation places
        "radius": radius,
        "limit": 1,  # Limit the result to the first hotel found
        "apiKey": geoapify_key  # Use your Geoapify API key from api_keys.py
    }
    
    # Set the location parameter to specify latitude and longitude
    params["location"] = f"{lat},{lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

hotel_df = hotel_df[hotel_df['Hotel Name'] != "No hotel found"]

hotel_df.reset_index(drop=True, inplace=True)

hotel_df['Hover'] = hotel_df.apply(lambda row: f"City: {row['City']}, Country: {row['Country']}, Hotel: {row['Hotel Name']}", axis=1)

map_plot = hotel_df.hvplot.points(
    x='Lng',
    y='Lat',
    geo=True,
    tiles='CartoLight',
    size=10,
    hover_cols=['Hover'],  
    title='Cities with Nearby Hotels'
)

map_plot


ValueError: cannot convert float NaN to integer

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hover)